In [5]:
#Importing required modules
import numpy as np
import pandas as pd
import keras
#Reading pixel values of images in the form of csv as a dataframe
data=pd.read_csv('C://Users//Jeshwanth//Desktop//all//fer2013//fer2013.csv')
#Seperating emotion and pixels values as Y_train and X_train respectively
X_train,Y_train=data['pixels'],data['emotion']
X_train=X_train.as_matrix()
Y_train=Y_train.as_matrix()
#Splitting pixls in the form of strings to lists
for i in range(X_train.size):
    X_train[i]=[int(s) for s in X_train[i].split(' ')]
#Converting list to np.array
X_train=np.array(X_train)
Y_train=np.array(Y_train)
#Converting each of the rows in X_train from strings to lists
for i in range(X_train.size):
    X_train[i]=np.array(X_train[i])
#Splitting X_train,Y_train  into (X_Train,X_test,X_private),(Y_Train,Y_test,Y_private)
X_Train=X_train[0:28710]
X_test=X_train[28710:32299]
X_private=X_train[32299:]
Y_Train=Y_train[0:28710]
Y_test=Y_train[28710:32299]
Y_private=Y_train[32299:]
#Reshaping the pixels into dimensions of images
for i in range(X_Train.size):
    X_train[i]=X_Train[i].reshape(48,48,1)
for i in range(X_test.size):
    X_test[i]=X_test[i].reshape(48,48,1)
for i in range(X_private.size):
    X_private[i]=X_private[i].reshape(48,48,1)
#One-hot encoding dependent variables
from keras.utils import to_categorical
Y_Train=to_categorical(Y_Train)
Y_test=to_categorical(Y_test)
Y_private=to_categorical(Y_private) 
#Data_augmentation
from keras.preprocessing.image import ImageDataGenerator
data_gen=ImageDataGenerator(rotation_range=20,
        width_shift_range=0.02,
        height_shift_range=0.02,
        horizontal_flip=True)
#Building the model and executing 
from keras.models import Sequential
from keras.layers import Conv2D,Dense,Flatten,MaxPooling2D,Dropout
model=Sequential()
model.add(Conv2D(32,kernel_size=3,activation='relu',input_shape=(48,48,1)))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2,padding='valid'))
model.add(Conv2D(64,kernel_size=3,activation='relu'))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size=2,padding='valid'))
model.add(Conv2D(128,kernel_size=3,activation='relu'))
model.add(Dropout(0.15))
model.add(MaxPooling2D(pool_size=2,padding='valid'))
model.add(Conv2D(256,kernel_size=3,activation='relu'))
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=2,padding='valid'))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.05))
model.add(Dense(32,activation='relu'))
model.add(Dense(7,activation='softmax'))
from keras import optimizers
opt=optimizers.Adam(lr=0.0001)
model.compile(optimizer=opt,metrics=['accuracy'],loss='categorical_crossentropy')
X_Train=np.concatenate([i for i in X_Train])
X_test=np.concatenate([i for i in X_test])
X_private=np.concatenate([i for i in X_private])
X_Train=X_Train.reshape(28710,48,48,1)
X_test=X_test.reshape(3589,48,48,1)
X_private=X_private.reshape(3588,48,48,1)
model.fit_generator(data_gen.flow(X_Train, Y_Train),steps_per_epoch=len(X_Train)//32, validation_data=(X_test, Y_test), epochs=150)


In [1]:
from keras.models import load_model

C:\Users\Jeshwanth\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Assing a no. to each of the expressions in a dictionary
em={0:"Angry", 1:'Disgust', 2:'Fear', 3:'Happy', 4:'Sad', 5:'Surprise',6:'Neutral'}
#Loading saved model
loaded_model=load_model('ema63.h5')
#Process to take input from webcam and detect the expression
import cv2
faceCascade = cv2.CascadeClassifier('C:\\Users\\Jeshwanth\\Desktop\\all\\haarcascade_frontalface_alt.xml')
# grab the reference to the webcam
vs = cv2.VideoCapture(0) #0 is webcam

# keep looping
while True:
 # grab the current frame
    ret, frame = vs.read()
    #print('Done')
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY);
    faces = faceCascade.detectMultiScale(gray)
    #print(len(faces))
    if(len(faces)>0):
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)
            temp=gray[y:y+h,x:x+w]
            img=cv2.resize(temp,(48,48))
            image=img.reshape(1,48,48,1)
            value=loaded_model.predict(image)
            font= cv2.FONT_HERSHEY_SIMPLEX
            bottomLeftCornerOfText = ((2*x+w)//2,y+h+1)
            fontScale= 1
            fontColor= (255,255,255)
            lineType = 2
            cv2.putText(frame,em[np.argmax(value)], bottomLeftCornerOfText, font, fontScale,fontColor,lineType)
            # show the frame to our screen
    cv2.imshow("Video", frame)
    #print(em[np.argmax(value)])
    key = cv2.waitKey(1) & 0xFF
    # if the 'q' key is pressed, stop the loop
    if key == ord("q"):
        break
vs.release()
cv2.destroyAllWindows()

In [20]:
#Testing accuracy on private test data
x=loaded_model.predict(X_private)
z=0
#print(x.size,Y_private.size)
for i in range(x.shape[0]):
    ze=np.argmax(Y_private[i])-np.argmax(x[i])
    if(ze==0):
        z=z+1
print(z/x.shape[0])

0.6401895206243032
